In [158]:
import os, sys
from importlib import reload

sys.path.append('/home/ubuntu/git/Genotyper/src/')
import utils.BitOperations as B

In [185]:
DEBUG=False
class alt_BitArray:
    def __init__(self, size, label='', data_packet=[]):
        self.label = label
        self.data_packet = data_packet
        self.max = size
        self.bf = 1 <<self.max+1
        self.debugLst = []
    def __getitem__(self, offset):
        assert(offset<self.max)
        return self.bf&(1 << offset)
    def __setitem__(self, offset, m):
        assert(offset<self.max)
        if DEBUG:
            if not(offset in self.debugLst):
                if m:
                    self.debugLst.append(offset)
                    self.debugLst.sort()
                else:
                    if offset in self.debugLst:
                        del self.debugLst[self.debugLst.index(offset)]
        if m:
            self.bf |= (1 << offset)
        else:
            self.bf &= ~(1 << offset)
    def count(self):
        count = 0
        for idx in range(self.max):
            if self.bf&(1 << idx):count+=1
        return count
    def debug_count(self):
        count = 0
        for idx in range(self.max):
            if self.bf&(1 << idx):count+=1
        if DEBUG:
            assert(len(self.debugLst)==count)
        return count
    def clearBits(self, offsetOrLst):
        if type(offsetOrLst)==type(0):
            offsetOrLst=[offsetOrLst]
        for offset in offsetOrLst:
            if DEBUG:
                if offset in self.debugLst:
                    del self.debugLst[self.debugLst.index(offset)]
            self[offset]=0
    def setBits(self, offsetOrLst):
        if type(offsetOrLst)==type(0):
            offsetOrLst=[offsetOrLst]
        for offset in offsetOrLst:
            self[offset]=1
    def __and__(self, other):
        assert (other.max == self.max)
        o = self.cleanCopy()
        o.bf = self.bf&other.bf
        return o
    def __or__(self, other):
        assert (other.max == self.max)
        o = self.cleanCopy()
        o.bf = self.bf|other.bf
        return o
    def __xor__(self, other):
        assert (other.max == self.max)
        o = self.cleanCopy()
        o.bf = self.bf^other.bf
        return o
    def cleanCopy(self):
        """Makes an empty copy or self"""
        return BitArray(self.max, label=self.label, data_packet=self.data_packet)
    def copy(self):
        """Makes an exact copy or self"""
        b = BitArray(self.max, label=self.label, data_packet=self.data_packet)
        b.array=copy.deepcopy(self.bf)
        b.debugLst=copy.deepcopy(self.debugLst)
        return b
    def setRange(self, start, end):
        for i in range(min(start, end), max(start, end), 1):
            self[i]=1
    def ANDFuse(self, other):
        #converts self to outcome of AND
        assert (other.max == self.max)
        self.bf=self.bf&other.bf
    def ORFuse(self, other):
        #converts self to outcome of OR
        assert (other.max == self.max)
        self.bf=self.bf|other.bf
    def XORFuse(self, other):
        #converts self to outcome of XOR
        assert (other.max == self.max)
        self.bf=self.bf^other.bf
    def export_bit_lst(self):
        return [1 if self[offset] else 0 for offset in range(self.max)]
    def export_bit_idxLst(self):
        l = []
        for offset in range(self.max):
            if self[offset]:
                l.append(offset)
        return l
    def __str__(self):
        sLst = []
        sLst.append('%s max_sz: %i count: %i'%(self.label, self.max, self.count()))
        sLst.append(self.bin_str())
        return '\n'.join(sLst)
    def Part(self):
        s = float(self.count())/self.max
        return s
    def bin_str(self):
        sLst = []
        b_str=bin(self.bf)[3:]
        return str_reverse(b_str)
    def test(self):
        bin_str=self.bin_str()
        print('export_bit_idxLst', self.export_bit_idxLst(), '\n',
              'filter(lambda i:self[i]', list(filter(lambda i:self[i]!=0, range(self.max))),'\n',
              'self.debugLst', self.debugLst, '\n',
              'count', self.debug_count(), '\n',
              'bin_str()', bin_str, '\n',
              'count(bin_str)', sum([int(x) for x in list(bin_str)]))
    
def str_reverse(s):
    s = list(s)
    s.reverse()
    return ''.join(s)

In [ ]:
a=1<<1000
type(a), a, a&(1 << 1000), bin(a)

In [202]:
import random, time
reload(B)
DEBUG=False
p=alt_BitArray(10000)
start=time.time()
for i in range(1000):
    idx=int(random.random()*(p.max-1))
    s=0 if random.random()<0.5 else 1
    if s:
        p[idx]=1
    else:
        p[idx]=1
    #p=p|p
    p.count()
    print('%i idx:%03d set:%i'%(i,idx,s), end='\r')
p.test()
print(p)
print('elapsed_time', time.time()-start)

export_bit_idxLst [1, 55, 60, 63, 64, 83, 84, 85, 86, 101, 108, 141, 145, 167, 181, 198, 201, 203, 209, 233, 244, 246, 255, 278, 294, 298, 302, 306, 324, 332, 333, 341, 354, 373, 404, 419, 433, 437, 439, 444, 449, 480, 490, 500, 506, 515, 516, 517, 518, 529, 535, 537, 553, 562, 596, 610, 633, 634, 642, 644, 684, 685, 719, 724, 745, 758, 771, 785, 801, 833, 842, 852, 853, 885, 914, 920, 922, 956, 976, 984, 1007, 1023, 1027, 1044, 1050, 1058, 1078, 1083, 1087, 1090, 1104, 1109, 1135, 1138, 1148, 1149, 1173, 1183, 1185, 1190, 1195, 1196, 1203, 1214, 1216, 1231, 1240, 1241, 1246, 1285, 1298, 1304, 1312, 1325, 1354, 1358, 1373, 1375, 1386, 1399, 1402, 1406, 1410, 1412, 1418, 1419, 1440, 1455, 1461, 1469, 1480, 1509, 1519, 1522, 1535, 1536, 1539, 1543, 1549, 1550, 1553, 1561, 1570, 1580, 1582, 1588, 1596, 1597, 1610, 1634, 1638, 1655, 1661, 1675, 1680, 1692, 1702, 1703, 1705, 1724, 1731, 1743, 1753, 1764, 1779, 1803, 1816, 1819, 1832, 1843, 1874, 1882, 1903, 1926, 1947, 1950, 1954, 1966, 197

In [ ]:
elapsed_time 5.734522104263306  no debug 5.7907938957214355
old version elapsed_time 1.5201425552368164 nodEbug 1.644566297531128

In [87]:

l=[0 for i in range(p.bf_size)]
for i in idxLst:l[i]=1
my_bstr=''.join([str(x) for x in l])

print(p.export_bit_idxLst(), '\n',
      p.bin(), '\n',
      my_bstr, '\n',
      p.t, '\n',
      p.count(), '\n',
      sum([int(x) for x in list(p.bin())]), '\n',
      list(filter(lambda i:p[i], range(p.max+1))),'\n',
      p.array)

[0, 3, 14, 22, 23, 24, 34, 52, 63, 67, 70, 73, 89, 95, 96, 99] 
 10010000000000100000001110000000001000000000000000001000000000010001001001000000000000000100000110010000000000000000000000000000 
 10010000000000100000001110000000001000000000000000001000000000010001001001000000000000000100000110010000000000000000000000000000 
 [0, 3, 14, 22, 23, 24, 34, 52, 63, 67, 70, 73, 89, 95, 96, 99] 
 16 
 16 
 [0, 3, 14, 22, 23, 24, 34, 52, 63, 67, 70, 73, 89, 95, 96, 99] 
 [9227875653691392009, 40835744328]


In [26]:
#[int(x) for x in list(p.bin())]
bstr=p.bin()
bstr, len(bstr), p.max

('000100000000001000000011100000000010000000000000000010010010010000000000000001000001100000000000000000',
 102,
 102)

In [42]:
i=p.array[0]
j=p.array[1]
bin(i), bin(j)

('0b10000000000000000010000000001110000000100000000001000',
 '0b100110000010000000000000001001001001')

In [28]:
p.array

[4503616836616200, 40835744329]

In [30]:
def count(int_type):
    count = 0
    while(int_type):
        count += (int_type & 1)
        int_type >>= 1
    return count

In [29]:
bin(p.array[0]), bin(p.array[1]), bin((p.array[0])&(p.array[1]))

('0b10000000000000000010000000001110000000100000000001000',
 '0b100110000010000000000000001001001001',
 '0b1000')

In [31]:
count((p.array[0])&(p.array[1]))

1

In [52]:
len('00010000000000100000001110000000001000000000000000000000000000001001001001000000000000000100000110000000000000000000000000000000')


128